# **House Price Prediction ML-Regression**

In [ ]:
#House Price Prediction ML-Regression

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Import Dataset and Import Libs**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error
warnings.filterwarnings('ignore')

Read Data

In [ ]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
train.head(10)

In [ ]:
train.info()

In [ ]:
plt.figure(figsize=[12, 9])
sns.heatmap(data=train.corr(), cmap='coolwarm', square=True)

**distribution check is normal or not**

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt', 'YearRemodAdd']
sns.pairplot(train[cols], size = 2.5)
plt.show()

In [ ]:
#Skewness & Kurtosis before log trans
print("Skewness: ", train['SalePrice'].skew())
print("Kurtosis: ", train['SalePrice'].kurt())

Log tranformation

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt', 'YearRemodAdd']
sns.pairplot(np.log1p(train[cols]), size = 2.5)
plt.show()

In [ ]:
print("Skewness: ", np.log1p(train['SalePrice']).skew())
print("Kurtosis: ", np.log1p(train['SalePrice']).kurt())

**we see that log-transformation helps us in reducing the skewness & kurtosis. So, I'm going with log-transformation**

**#Check For Missing values**

In [ ]:
#Check For Missing values
total = train.isna().sum().sort_values(ascending=False)
percent = (train.isna().sum()/train.isna().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

**Features like 'PoolQC', 'MiscFeature', 'Alley' , 'Fence' need to be dropped because they contain a lot of missing values which we cannot fill with the help of existing data. It will be too much risk. Also, there are some features having same number of missing values like 'GarageX', 'BsmtX', 'MasVnrX'. Regarding 'MasVnrArea' and 'MasVnrType', we can consider that these variables are not essential. Furthermore, they have a strong correlation with 'YearBuilt' and 'OverallQual' which are already considered. Thus, I think we will not lose information if we delete 'MasVnrArea' and 'MasVnrType'**

> **Filling the missing Data**

In [ ]:
train = train.drop((missing_data[missing_data['Total'] > 1]).index, 1)
train = train.drop(train.loc[train['Electrical'].isna()].index)
train.isna().sum().max()

Check Any Outliers in Data

In [ ]:
saleprice_scaled = StandardScaler().fit_transform(train['SalePrice'][:, np.newaxis])
low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
high_range= saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]
print('outer range (low) of the distribution:')
print(low_range)
print('\nouter range (high) of the distribution:')
print(high_range)

**We see that low-range values are close to 0 but high-range don't have the same case as low-range ones. We have to be more careful with those two 7's in high-range. I'll remove them next.**

In [ ]:
data = pd.concat([train['SalePrice'], train['GrLivArea']], axis=1)
data.plot(kind='scatter', x='GrLivArea', y='SalePrice')

In [ ]:
train.sort_values(by='GrLivArea', ascending=False)[:2]
train = train.drop(train[train['Id'] == 1299].index)
train = train.drop(train[train['Id'] == 524].index)

In [ ]:
data = pd.concat([train['SalePrice'], train['GrLivArea']], axis=1)
data.plot(kind='scatter', x='GrLivArea', y='SalePrice')

**Perform Log transformation --before**

In [ ]:
sns.distplot(train['SalePrice'], fit=norm)

In [ ]:
stats.probplot(train['SalePrice'], plot=plt)
plt.show()

**Log transformation -after**

In [ ]:
train['SalePrice'] = np.log1p(train['SalePrice'])

In [ ]:
sns.distplot(train['SalePrice'], fit=norm)

In [ ]:
stats.probplot(train['SalePrice'], plot=plt)
plt.show()

# ** Feature Engineering**

**I'll create a variable that can get the effect of having or not having basement (binary variable). Then, I'll do a log transformation to all the non-zero observations, ignoring those with value zero.**

In [ ]:
train['HasBsmt'] = pd.Series(len(train['TotalBsmtSF']), index=train.index)
train['HasBsmt'] = 0
train.loc[train['TotalBsmtSF']>0, 'HasBsmt'] = 1

In [ ]:
train.loc[train['HasBsmt'] == 1, 'TotalBsmtSF'] = np.log(train['TotalBsmtSF'])

In [ ]:
sns.distplot(train[train['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm)

**Categorical features**

**Training and Test data**

In [ ]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
x = train.drop('Id', axis=1)
y = train['SalePrice']

**Ensuring same columns in both train & test dataset**

In [ ]:
x = x.drop(x.columns.difference(test.columns), axis =1)
test = test.drop(test.columns.difference(x.columns), axis = 1)
if len(x.columns) == len(test.columns):
       print('Train and test dataset are equal')

**Drop features**

In [ ]:
high_corr_feats = [m for m in ['GarageCars', 'GarageArea', 'TotRmsAbvGrd', 'GrLivArea', 'TotalBsmtSF', '1stflrSF', 'FullBath', 'GrLivArea', 'YearRemodAdd'] if m in x.columns][::2]

In [ ]:
x = x.drop(high_corr_feats, axis=1)
test = test.drop(high_corr_feats, axis=1)

**Training and Test Split **

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=101)

In [ ]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
x_train = imp_mean.fit_transform(x_train) 
x_test = imp_mean.fit_transform(x_test)

# MODELS

# Linear Regression

In [ ]:
#MODELS
lr = LinearRegression()
lr.fit(x_train, y_train)
lr_pred = lr.predict(x_test)
lr_score = r2_score(y_test, lr_pred)
lr_mse = mean_squared_error(y_test, lr_pred)
print("Linear Regression testing accuracy r2_score: ", lr_score*100)
print("Linear Regression MSE: ", lr_mse)

# Ridge

In [ ]:
rcv = RidgeCV(alphas=(0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0), cv=8)
rcv.fit(x_train, y_train)
rcv_pred = rcv.predict(x_test)
rcv_score = r2_score(y_test, rcv_pred)
rcv_mse = mean_squared_error(y_test, rcv_pred)
print("Ridge Regression testing accuracy r2_score: ", rcv_score*100)
print("Ridge Regression MSE: ", rcv_mse)

# LassoCV

In [ ]:
lcv = LassoCV(eps=0.00001, n_alphas=10000)
lcv.fit(x_train, y_train)
lcv_pred = lcv.predict(x_test)
lcv_score = r2_score(y_test, lcv_pred)
lcv_mse = mean_squared_error(y_test, lcv_pred)
print("Lasso Regression testing accuracy r2_score: ", lcv_score*100)
print("Lasso Regression MSE: ", lcv_mse)

# XGB Regressor

In [ ]:
xgb = XGBRegressor()
xgb.fit(x_train, y_train)
xgb_pred = xgb.predict(x_test)
xgb_score = r2_score(y_test, xgb_pred)
xgb_mse = mean_squared_error(y_test, xgb_pred)
print("XGB Regressor testing accuracy r2_score: ", xgb_score*100)
print("XGB Regressor MSE: ", xgb_mse)

# CatBoostRegressor

In [ ]:
cbr = CatBoostRegressor(verbose=False, depth=8, learning_rate=0.001)
cbr.fit(x_train, y_train)
cbr_pred = cbr.predict(x_test)
cbr_score = r2_score(y_test, cbr_pred)
cbr_mse = mean_squared_error(y_test, cbr_pred)
print("CatBoostRegressor testing accuracy r2_score: ", cbr_score*100)
print("CatBoostRegressor MSE: ", cbr_mse)

In [ ]:
r2_scores = [lr_score*100, rcv_score*100, lcv_score*100, xgb_score*100, cbr_score*100]
mse = [lr_mse, rcv_mse, lcv_mse, xgb_mse, cbr_mse]
model = ['Linear Reg', 'Ridge Reg', 'Lasso Reg', 'XGB Reg', 'CatBoost Reg']
model_showdown = pd.DataFrame({'Model': model, 'r2_score': r2_scores, 'mse': mse})

In [ ]:
model_showdown

In [ ]:
plt.figure(figsize=[15, 8])
plt.subplot(1, 2, 1)
sns.barplot(x='Model', y='r2_score', data=model_showdown)
plt.subplot(1, 2, 2)
sns.barplot(x='Model', y='mse', data=model_showdown)
plt.tight_layout()

In [ ]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
test = imp_mean.fit_transform(test) 

# I'll go with Ridge Regression as I think it performs better.

In [ ]:
end = rcv.predict(test)

In [ ]:
end

**Values are in log so I'll convert them back to their original scale.**

In [ ]:
end = np.exp(end)

In [ ]:
end

# Submission

In [ ]:
sample = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [ ]:
ID = sample['Id']

In [ ]:
submit = pd.DataFrame({'Id': ID, 'SalePrice': end})
submit

In [ ]:
submit.to_csv('submission.csv', index=False)